In [71]:
import numpy as np
import gradio as gr
import requests
from PIL import Image
from openai import OpenAI


def run_demo():
    """Setup the app interface and launch it."""
    with gr.Blocks() as app:

        gr.Markdown('# Mental Health Nudging with Generative AI Demo')
        with gr.Row():

            # input features
            with gr.Column(scale=1):

                # demographics
                gender = gr.Radio(label='Gender', choices=['Male', 'Female', 'Non-Binary'])
                age = gr.Slider(label='Age', minimum=18, maximum=80, step=1)

                # symptoms
                symptoms = gr.CheckboxGroup(label='Symptoms',
                                            choices=['Anxiety', 'Depression', 'Tiredness', 'Apathy'])

                # interests
                likes = gr.Textbox(label='Liked Activities',
                                   placeholder='Comma-separated list of likes...')
                dislikes = gr.Textbox(label='Disliked Activities',
                                      placeholder='Comma-separated list of dislikes...')

                # submit button
                submit_button = gr.Button('Generate Nudge')

            # resulting nudge
            with gr.Column(scale=1):
                nudge_image = gr.Image(label='Nudge Image')
                nudge_message = gr.Textbox(label='Nudge Message')

        # submit parameters for nudge generation
        inputs = [gender, age, symptoms, likes, dislikes]
        outputs = [nudge_image, nudge_message]
        submit_button.click(fn=generate, inputs=inputs, outputs=outputs)

    # launch the app
    gr.close_all()
    app.queue(default_concurrency_limit=None)
    app.launch()


def generate(gender, age, symptoms, likes, dislikes):
    """Generate nudging image and message for the given person."""
    # construct description of the person
    desc = f'A {age} year old {"man" if gender=="Male" else "woman" if gender=="Female" else "person"}.'
    if likes:
        desc += f' They like {likes}.'
    if dislikes:
        desc += f' They do not like {dislikes}.'
    if symptoms:
        desc += f' They have the following symptoms: {", ".join(symptoms)}.'
    else:
        desc += f' They do not have any symptoms.'

    # generate nudge for the person
    nudge_message = generate_nudge_message(desc)
    nudge_image = generate_nudge_image(desc, nudge_message, random=False)

    return nudge_image, nudge_message


def generate_nudge_message(desc):
    """Generate a message for a given person."""
    print(f'{desc=}')
    system_prompt = 'You are a behavioral content writer. You job is to write short '\
                  + 'motivational messages to help people with their mental health. '\
                  + 'Messages should be at most a paprgraph long.'
    user_prompt = f'Write a short inspirational message for the person with the following description:\n\n{desc}'
    messages = [{'role': 'system', 'content': f'{system_prompt}'},
                {'role': 'user', 'content': f'{user_prompt}'}]
    completion = client.chat.completions.create(messages=messages, model='gpt-3.5-turbo')
    nudge_message = completion.choices[0].message.content
    return nudge_message


def generate_nudge_image(desc, nudge_message, random=True):
    """Generate an image for a given person and message."""
    if random:
        return Image.fromarray(np.random.randint(0, 255, (100, 100, 3), dtype='uint8'), 'RGB')

    prompt = 'Illustrate an inspirational message to help a person with their mental health. '\
           + f'The person is described as follows:\n\n{desc}\n\n'\
           + f'The image should align with the following message:\n\n{nudge_message}'

    print(f'{prompt=}')
    response = client.images.generate(prompt=prompt, model='dall-e-2', style='vivid', quality='standard')
    nudge_image = Image.open(requests.get(response.data[0].url, stream=True).raw)

    return nudge_image


if __name__ == '__main__':
    client = OpenAI()
    run_demo()


Running on local URL:  http://127.0.0.1:7899

To create a public link, set `share=True` in `launch()`.


desc='A 42 year old woman. They like cats, cooking. They do not like outside. They have the following symptoms: Anxiety.'
prompt='Illustrate an inspirational message to help a person with their mental health. The person is described as follows:\n\nA 42 year old woman. They like cats, cooking. They do not like outside. They have the following symptoms: Anxiety.\n\nThe image should align with the following message:\n\nDear friend, I understand that anxiety can sometimes feel overwhelming, but remember, you are stronger than you think. Just like cats have their own safe spaces, create a sanctuary within yourself where you can find peace. Embrace the comforting art of cooking, allowing it to ignite your passion and bring you joy. Though the outside may seem daunting, know that within your soul lies the power to conquer any fear. Be kind to yourself, take small steps, and remember that you are capable of remarkable things.'
